In [1]:
!pip3 install sklearn

In [2]:
import pandas as pd
import numpy as np
import re
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
stop_words = stopwords.words('english')

In [3]:
project_df = pd.read_csv('project_hub_summary.csv')
result_df = pd.DataFrame(columns=['Precision', 'Recall', 'F1'])

In [4]:
project_df.head()

,catigory,link,text,Text_Rank,LEAD3,BART,text_len,LEAD3_len,LEAD3_len_percent,Text_Rank_len,Text_Rank_len_percent,BART_len,BART_len_percent
0,audio-sound,https://create.arduino.cc//projecthub/buzzandy...,My niece is getting into music so I to make he...,Introduction This project was pretty simple an...,My niece is getting into music so I to make he...,My niece is getting into music so I to make h...,262.0,37.0,0.141221,85.0,0.324427,63.0,0.240458
1,audio-sound,https://create.arduino.cc//projecthub/shajeeb/...,This project is for making a band audio music ...,Main of this frequency spectrum analyzer easil...,This project is for making a band audio music ...,This project is for making a band audio music...,810.0,47.0,0.058025,157.0,0.193827,61.0,0.075309
2,audio-sound,https://create.arduino.cc//projecthub/ericBcre...,The idea seen some I thought it would be cool ...,fooling around with a spinning and version exc...,The idea seen some I thought it would be cool ...,The idea seen some I thought it would be cool...,590.0,60.0,0.101695,192.0,0.325424,51.0,0.086441
3,audio-sound,https://create.arduino.cc//projecthub/lindsi87...,To begin I should fill you in that I am comple...,and setting the frequency for each of the pian...,To begin I should fill you in that I am comple...,The basic for both the main and the individua...,789.0,64.0,0.081115,90.0,0.114068,45.0,0.057034
4,audio-sound,https://create.arduino.cc//projecthub/jrance/s...,Learning electronics not only can be fan but i...,I used to be one of those thinking I would nev...,Learning electronics not only can be fan but i...,Learning electronics not only can be fan but ...,219.0,62.0,0.283105,86.0,0.392694,68.0,0.310502


In [5]:
print(project_df['catigory'].value_counts())

internet-of-things-bt-wireless    107
wearables-health-fitness          102
lab-stuff                         101
motors-robotics                    99
home-automation                    94
gadgets-games-toys                 87
audio-sound                        87
lights-leds                        82
sensors-environment                82
screens-displays                   74
Name: catigory, dtype: int64


In [6]:
def prepare_data(df):
    train_df = df.sample(frac = 0.8, random_state = 100)
    test_df = df.drop(train_df.index).reset_index(drop=True)
    train_df = train_df.reset_index(drop = True)
    print('{},{}'.format(str(len(train_df)), str(len(test_df))))
    return train_df, test_df

In [7]:
MAX_FEATURE = 1000

In [8]:
train_df, test_df = prepare_data(project_df)
print(train_df['catigory'].value_counts())
print(test_df['catigory'].value_counts())

732,183
internet-of-things-bt-wireless    89
wearables-health-fitness          85
motors-robotics                   78
lab-stuff                         75
home-automation                   75
gadgets-games-toys                73
lights-leds                       66
audio-sound                       66
sensors-environment               64
screens-displays                  61
Name: catigory, dtype: int64
lab-stuff                         26
audio-sound                       21
motors-robotics                   21
home-automation                   19
sensors-environment               18
internet-of-things-bt-wireless    18
wearables-health-fitness          17
lights-leds                       16
gadgets-games-toys                14
screens-displays                  13
Name: catigory, dtype: int64


In [9]:
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()
len(word_embeddings)

400000

In [10]:
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

def sentence_glove(text_list):
    sentence_glove = []
    for text in text_list:
        clean_sentences = re.sub("[^a-zA-Z]", " ", text)
        clean_sentences = clean_sentences.lower()
        clean_sentences = remove_stopwords(clean_sentences.split())
        tokens = clean_sentences.split()
        v = []
        for i in range(100):
            if i < len(tokens):
                v.append(word_embeddings.get(tokens[i], np.zeros((100,))))
            else:
                v.append(np.zeros((100,)))
#         v = sum([word_embeddings.get(w, np.zeros((100,))) for w in clean_sentences.split()])/(len(clean_sentences.split())+0.001)
        sentence_glove.append(np.array(v).flatten())
    return sentence_glove

In [11]:
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, recall_score, precision_score, confusion_matrix, classification_report
def report_results(A, B):
    A_name = A.name
    B_name = B.name
    
    df = pd.DataFrame({'A':A,
                       'B':B})
    df = df.dropna()
    A = df['A']
    B = df['B']
    
    acc = accuracy_score(B, A)
    f1 = f1_score(B, A, average='weighted')
    prec = precision_score(B, A, average='weighted')
    rec = recall_score(B, A, average='weighted')

    print('accuracy: %0.4f \nprecision: %0.4f \nrecall: %0.4f \nF1 score: %0.4f \n' % (acc, prec, rec, f1))
    print(confusion_matrix(B, A))
    print(classification_report(B, A))
    return prec, rec, f1

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

In [13]:
x_train = sentence_glove(train_df['text'])
y_train = train_df['catigory']
x_test = sentence_glove(test_df['text'])
y_test = test_df['catigory']

    
model_lc = LogisticRegression()
model_lc.fit(x_train, y_train)
test_df['LR_Prediction'] = model_lc.predict(x_test)
print('Logistic Regression\n')
prec, rec, f1 = report_results(test_df['LR_Prediction'], test_df['catigory'])
incorrect = test_df[test_df['LR_Prediction'] != test_df['catigory']]
print('incorrect: ' + str(len(incorrect)))
result_df.loc['Original_LR'] = [prec, rec, f1]    

Logistic Regression

accuracy: 0.3279 
precision: 0.3132 
recall: 0.3279 
F1 score: 0.3158 

[[ 7  3  3  2  2  0  2  1  0  1]
 [ 0  4  2  0  2  1  3  1  1  0]
 [ 1  3  1  6  3  0  2  0  2  1]
 [ 0  3  0  6  5  1  1  0  1  1]
 [ 2  1  3  4  6  2  3  1  3  1]
 [ 1  3  0  0  2  7  1  1  1  0]
 [ 0  1  1  3  1  1 12  0  1  1]
 [ 4  3  1  1  3  1  0  0  0  0]
 [ 0  1  0  2  0  2  3  0  7  3]
 [ 0  2  1  0  1  1  1  0  1 10]]
                                precision    recall  f1-score   support

                   audio-sound       0.47      0.33      0.39        21
            gadgets-games-toys       0.17      0.29      0.21        14
               home-automation       0.08      0.05      0.06        19
internet-of-things-bt-wireless       0.25      0.33      0.29        18
                     lab-stuff       0.24      0.23      0.24        26
                   lights-leds       0.44      0.44      0.44        16
               motors-robotics       0.43      0.57      0.49        21

/Users/yiwenshi/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [14]:
x_train = sentence_glove(train_df['LEAD3'])
y_train = train_df['catigory']
x_test = sentence_glove(test_df['LEAD3'])
y_test = test_df['catigory']
    
model_lc = LogisticRegression()
model_lc.fit(x_train, y_train)
test_df['LR_Prediction'] = model_lc.predict(x_test)
print('Logistic Regression\n')
prec, rec, f1 = report_results(test_df['LR_Prediction'], test_df['catigory'])
incorrect = test_df[test_df['LR_Prediction'] != test_df['catigory']]
print('incorrect: ' + str(len(incorrect)))
result_df.loc['LEAD3_LR'] = [prec, rec, f1]    

Logistic Regression

accuracy: 0.2842 
precision: 0.2777 
recall: 0.2842 
F1 score: 0.2781 

[[ 5  4  0  1  5  2  1  0  0  3]
 [ 2  2  1  2  0  1  3  1  1  1]
 [ 1  5  3  1  3  0  4  0  1  1]
 [ 0  2  1  8  2  0  3  0  1  1]
 [ 2  2  3  2  3  0  2  3  5  4]
 [ 2  2  0  1  0  7  1  0  2  1]
 [ 1  0  2  2  3  1  8  1  1  2]
 [ 3  0  3  0  2  0  1  3  1  0]
 [ 0  1  4  2  2  1  2  3  3  0]
 [ 0  0  0  0  0  3  1  0  3 10]]
                                precision    recall  f1-score   support

                   audio-sound       0.31      0.24      0.27        21
            gadgets-games-toys       0.11      0.14      0.12        14
               home-automation       0.18      0.16      0.17        19
internet-of-things-bt-wireless       0.42      0.44      0.43        18
                     lab-stuff       0.15      0.12      0.13        26
                   lights-leds       0.47      0.44      0.45        16
               motors-robotics       0.31      0.38      0.34        21

/Users/yiwenshi/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [15]:
x_train = sentence_glove(train_df['Text_Rank'])
y_train = train_df['catigory']
x_test = sentence_glove(test_df['Text_Rank'])
y_test = test_df['catigory']
    
model_lc = LogisticRegression()
model_lc.fit(x_train, y_train)
test_df['LR_Prediction'] = model_lc.predict(x_test)
print('Logistic Regression\n')
prec, rec, f1 = report_results(test_df['LR_Prediction'], test_df['catigory'])
incorrect = test_df[test_df['LR_Prediction'] != test_df['catigory']]
print('incorrect: ' + str(len(incorrect)))
result_df.loc['Text_Rank_LR'] = [prec, rec, f1]    

Logistic Regression

accuracy: 0.2295 
precision: 0.2458 
recall: 0.2295 
F1 score: 0.2220 

[[2 2 0 0 5 1 3 3 0 5]
 [0 3 2 3 1 1 2 1 0 1]
 [0 2 2 4 1 2 0 1 2 5]
 [1 1 0 4 2 1 4 1 2 2]
 [5 2 0 9 5 1 1 1 0 2]
 [1 2 1 0 0 7 1 1 0 3]
 [0 1 0 3 5 1 8 1 0 2]
 [1 1 1 1 1 1 2 3 0 2]
 [0 3 1 1 2 0 3 1 2 5]
 [2 1 0 2 0 0 5 1 0 6]]
                                precision    recall  f1-score   support

                   audio-sound       0.17      0.10      0.12        21
            gadgets-games-toys       0.17      0.21      0.19        14
               home-automation       0.29      0.11      0.15        19
internet-of-things-bt-wireless       0.15      0.22      0.18        18
                     lab-stuff       0.23      0.19      0.21        26
                   lights-leds       0.47      0.44      0.45        16
               motors-robotics       0.28      0.38      0.32        21
              screens-displays       0.21      0.23      0.22        13
           sensors-environm

/Users/yiwenshi/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [16]:
x_train = sentence_glove(train_df['BART'])
y_train = train_df['catigory']
x_test = sentence_glove(test_df['BART'])
y_test = test_df['catigory']
    
    
model_lc = LogisticRegression()
model_lc.fit(x_train, y_train)
test_df['LR_Prediction'] = model_lc.predict(x_test)
print('Logistic Regression\n')
prec, rec, f1 = report_results(test_df['LR_Prediction'], test_df['catigory'])
incorrect = test_df[test_df['LR_Prediction'] != test_df['catigory']]
print('incorrect: ' + str(len(incorrect)))
result_df.loc['BART_LR'] = [prec, rec, f1]    

Logistic Regression

accuracy: 0.3279 
precision: 0.3690 
recall: 0.3279 
F1 score: 0.3350 

[[ 8  3  2  4  1  2  0  1  0  0]
 [ 0  5  5  0  0  2  1  1  0  0]
 [ 3  2  6  2  2  1  3  0  0  0]
 [ 1  2  3  6  2  0  2  1  0  1]
 [ 0  1  1  7  7  2  3  3  1  1]
 [ 0  2  1  1  1  6  1  2  2  0]
 [ 0  2  2  2  4  0 10  0  0  1]
 [ 0  1  4  2  5  0  0  1  0  0]
 [ 0  2  5  2  1  1  1  2  3  1]
 [ 0  2  1  0  1  2  1  0  2  8]]
                                precision    recall  f1-score   support

                   audio-sound       0.67      0.38      0.48        21
            gadgets-games-toys       0.23      0.36      0.28        14
               home-automation       0.20      0.32      0.24        19
internet-of-things-bt-wireless       0.23      0.33      0.27        18
                     lab-stuff       0.29      0.27      0.28        26
                   lights-leds       0.38      0.38      0.38        16
               motors-robotics       0.45      0.48      0.47        21

/Users/yiwenshi/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [17]:
result_df

,Precision,Recall,F1
Original_LR,0.313185,0.327869,0.315849
LEAD3_LR,0.277677,0.284153,0.278099
Text_Rank_LR,0.245760,0.229508,0.221994
BART_LR,0.369014,0.327869,0.334955
